# Black Holes At Home Tutorial: Python to Visit

## Author: Tyndale Stutzman

## This tutorial demonstrates how to plot Synthetic Data in Visit via Python

## Introduction: 

By using Python in conjunction with the VisIt visualization software, plots, rendering, and preferences can all be automated with the visit python library; either through the terminal, or a script. This both saves time and provides access to key attributes of plots, that would otherwise be restrictively displayed in the GUI, such as assigning variables to columns of a CSV file. This tutorial provides the necessary documentation to begin plotting synthetic data in VisIt directly from python.

### Troubleshooting

Troubleshooting issues with the VisIt python library can be cumbersome given the software's generic name, however, this PDF provides axhaustive documentation of nearly all functions in the library. If an issue persists, try assigning a letter to a function and calling the letter. For isntance, to display all possible arguments of `visit.MeshAttributes()` type `m = visit.MeshAttributes()` followed by `print(m)` into the python interpreter.

### For 302

If you are running this on the Etienne Work Computer in Room 302, skip to  \[Step [4](#run)\]


###### *This notebook is very much in progress

<a id='toc'></a>
# Table of Contents:
$$\label{toc}$$
<!--these buttons aren't functioning properly so maybe figure that out-->
1. [Step 1](#installing_visit): Installing VisIt
    1. [1.1](#windows): Windows
    1. [1.2](#osx/gnu): OSX/GNU
1. [Step 2](#installing_modules): Installing necessary python Modules
1. [Step 3](#gen): Generate synthetic data
    1. [3.1](#synth): Synthetic black hole merger data
    1. [3.2](#mesh): Create sphere mesh
1. [Step 4](#run): Python to VisIt
    1. [4.1](#load): Load the VisIt module
    1. [4.2](#database&plot): Load the database and plot
    1. [4.3](#adjust): Adjust view
    1. [4.4](#translate): Translate objects
1. [Step 5](#tune): Fine Tuning Visit
1. [Step 6](#conclusion): Conclusion

<a id='installing_visit'></a>
# Step 1: Installing VisIt
$$\label{installing_visit}$$


<a id='installing_modules'></a>
# Step 2: Installing necessary python Modules
$$\label{installing_modules}$$

In order to generate the synthetic data, a few dependencies must first be installed. For OSX/GNU users, enter the following code in the terminal:

In [ ]:
pip3 install install scipy matplotlib math numpy trimesh

For windows users, type the following:

In [ ]:
pip install scipy 
pip install matplotlib 
pip install math 
pip install numpy 
pip install trimesh

<a id='gen'></a>
# Step 3: Generate synthetic data
$$\label{gen}$$

With the necessary modules installed, the following python script will produce a `.csv` file containing the coordinate points of a single spiralling black hole. It should be noted that this script has been modified to exclusively save the data of one black hole to the file rather than both. As the simulation becomes more comprehensive, the associated data will include more variables. 

Further, in order for visit to recognize the existance of an object with coordinates, the object itself must be created. In this tutorial a mesh is used to plot a sphere with `n` number of points signifying its resolution. 

<a id='synth'></a>
## 3.1: Generate synthetic black hole merger data
$$\label{synth}$$

This python script uses the Error Function as a close approximation to the behavior of a spiralling black hole. It samples over 1000 internvals to produce three columns containing, time, the x coordinate, and the y coordinate of the imaginary black hole. The data are then saved in their respective columns to a `.csv` file labeled 'synthetic_data_BHtxyOnly.csv.'

In [ ]:
from scipy import special
import matplotlib as plt
from math import pi,cos,sin

# Error function
def ERF(x,x0,w):
    return 0.5 * (scipy.special.erf(((x)-(x0))/(w)) + 1.0)

destination_directory = r"./"
t_final = 2000
num_data_pts = 1000
deltat = (t_final)/num_data_pts
BH1radius = 0.25
BH2radius = 0.25
orbital_period = 225
omega = 2*pi/orbital_period
  
with open(destination_directory + "synthetic_data_BH1txyOnly.csv", "w") as file:
    file.write("time BH1x BH1y BH2x BH2y\n")
    for i in range(num_data_pts):
        time = deltat * i 
        orbital_separation = ERF(time, 1000, -500) 
        # BH1 coords
        BH1x = 0.5 * orbital_separation * cos(omega * time)
        BH1y = 0.5 * orbital_separation * sin(omega * time)
        BH1z = 0
        # BH2 coords
        BH2x = -0.5 * orbital_separation * cos(omega * time)
        BH2y = -0.5 * orbital_separation * sin(omega * time)
        BH2z = 0
        
        # Typecast
        outstr = str(time) + "," + str(BH1x) + "," + str(BH1y) + "\n"
        file.write(outstr)


<a id='mesh'></a>
## 3.2: Create sphere mesh
$$\label{mesh}$$

As mentioned above, VisIt needs an object to associate with the coordinates of an imaginary black hole. This is accomplished by plotting a sphere in python with resultion determined by `n` that is then saved as an `.obj` file. VisIt will ultimately treat this file as the alleged black hole. 

In [ ]:
import numpy as np
import trimesh

# Create parametric surface
def sphere(r, n_points):
    u = np.linspace(0, 2 * np.pi, n_points)
    v = np.linspace(0, np.pi, n_points)
    x = r * np.outer(np.cos(u), np.sin(v))
    y = r * np.outer(np.sin(u), np.sin(v))
    z = r * np.outer(np.ones(np.size(u)), np.cos(v))
    return x, y, z
# Save the surface as .obj file
def save_sphere_obj(x, y, z, filename):
    faces = np.array([])
    for i in range(x.shape[0] - 1):
        for j in range(x.shape[1] - 1):
            a = i * x.shape[1] + j
            b = i * x.shape[1] + j + 1
            c = (i + 1) * x.shape[1] + j
            d = (i + 1) * x.shape[1] + j + 1
            faces = np.append(faces, [a, b, c])
            faces = np.append(faces, [b, c, d])
    faces = faces.reshape((-1, 3)).astype(np.int64)
    mesh = trimesh.Trimesh(vertices=np.c_[x.flatten(), y.flatten(), z.flatten()], faces=faces)
    mesh.export(filename, file_type='obj')

# Set resolution to 50pts
n = 50
x, y, z = sphere(1, n)
save_sphere_obj(x, y, z, 'sphere_50pts.obj')

<a id='run'></a>
# Step 4: Python to Visit
$$\label{run}$$

Now that the synethetic data and sample mesh are in the local directory, Python can be used to quickly render a basic visualization. 

###### *If you skipped to step 4, the necessary files are already in your directory

<a id='load'></a>
## 4.1: Load the VisIt module
$$\label{load}$$
This directory is particular to the grad room computer, otherwise your local directory will be different. 

In [ ]:
# Set directory. This step is only necessary
#   if you're not running in jupyter. 
cd
cd Documents/VisIt/tutorial1

# Launch python...
ipython 

In [ ]:
# Python
import sys
sys.path.append("/usr/local/3.3.1/linux-x86_64/lib/site-packages") 
import visit
visit.Launch()
import visit # second import is 'import'ant. ha, get it? 
v = visit # allows for simplifying everything to v.command

<a id='database&plot'></a>
## 4.2: Load database and add plot
$$\label{database&plot}$$

The database here is also conditional to the directory. I'll add the obj gen script soon.

In [ ]:
v.OpenDatabase("sphere_50pts.obj") #
v.AddPlot("Mesh", "OBJMesh")
# m = visit.MeshAttributes()
# print(p) 
v.DrawPlots()

# axis3D.bboxLocation(xmin, xmax, ymin, ymax, zmin, zmax)

AnnotationAtts = v.AnnotationAttributes()
AnnotationAtts.axes3D.bboxLocation = (-10, 10, -10, 10, -3, 3)
v.SetAnnotationAttributes(AnnotationAtts)
View3dAtts = v.SetView3D()
v.SetView3D(View3DAtts) 

'''
# Add second sphere
v.AddPlot("Mesh", "OBJMesh")
# Move said sphere
v.AddOperator("Transform")
t = v.GetOperatorOptions(0)
print(t)
'''

<a id='tune'></a>
# Step 5: Adjust view
$$\label{tune}$$

Here the user has a plethora of options for visualization preference. The following code demonstrates how to adjust the current view to simply appear less abrasive. 
